## THE CIRCUIT CONFIGURATION

In [1]:
# check if notebook is in colab
try:
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

import os
import json
import ezkl
import pandas as pd

import torch
from torchvision import transforms
from PIL import Image

import logging
import os
import sys
import sys
import time
from datetime import datetime

current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(project_root)
from training.defineSNN import MODSiameseBambooNN


# uncomment for more descriptive logging
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.INFO)


# Specify all the files we need
model_path = os.path.join('ezkl-outputs/snn.onnx')
compiled_model_path = os.path.join('ezkl-outputs/snn.ezkl')
pk_path = os.path.join('ezkl-outputs/pk.key')
vk_path = os.path.join('ezkl-outputs/vk.key')
settings_path = os.path.join('ezkl-outputs/settings.json')
srs_path = os.path.join('ezkl-outputs/kzg.srs')
data_path = os.path.join('ezkl-outputs/input.json')
cal_path = os.path.join('ezkl-outputs/cal_data.json')
##
sol_code_path = os.path.join('../contracts/Verifier.sol')
abi_path = os.path.join('ezkl-outputs/Verifier.abi')
proof_path = os.path.join('ezkl-outputs/proof.json')




In [2]:
### Assign benchmark test name each time a new test is executed to save its results into a file

timestamp = datetime.now().strftime("%m_%d_%H:%M")
test_name = f"test_2___{timestamp}"
test_time_and_size_path = os.path.join(f'../benchmark/{test_name}_time_and_size.txt')
#test_gas_path = os.path.join(f'../benchmark/{test_name}_gas.html')

print(test_name)

test_2___01_08_18:58


In [3]:
### Load the image pair as input ready for inference
class SiameseImageLoader:
    def __init__(self, transform=None):
        self.transform = transform

    def load_and_transform_pair(self, image1_path, image2_path):
        img1 = Image.open(image1_path).convert('L')
        img2 = Image.open(image2_path).convert('L')

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        img1 = img1.to(device)
        img2 = img2.to(device)

        return img1, img2

transform = transforms.Compose([
    transforms.Resize((10, 10)),
    transforms.ToTensor(),
])

# Example inputs (TrueNegative, TruePositive, FalseNegative)
# (FalseNegative were produced with a growing rate of bamboo higher than what the model was trained on)
siamese_loader = SiameseImageLoader(transform)
image1_path = "TestValid_t1.jpeg"
image2_path = "TestValid_t0.jpeg"
img1, img2 = siamese_loader.load_and_transform_pair(image1_path, image2_path) # Separate tensors for each image
img1 = img1.unsqueeze(0)
img2 = img2.unsqueeze(0)

print(img1.size())

torch.Size([1, 1, 10, 10])


In [4]:
# Load the trained nn model and make inference on test inputs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_model = MODSiameseBambooNN().to(device)
loaded_model.load_state_dict(torch.load('../training/trained_simplesnn_lr_0.01.pth', map_location=torch.device('cpu')))
loaded_model.eval()

with torch.no_grad():
    finalout = loaded_model(img1, img2)
    predicted_label = (torch.sigmoid(finalout) > 0.5).item()

print(f'Same class probability: {torch.sigmoid(finalout)}')
print(f'Predicted Label: {predicted_label}')

# Calculate the total number of parameters
total_params = sum(p.numel() for p in loaded_model.parameters())
print(f'Total Parameters: {total_params}')

Same class probability: tensor([[0.6949]])
Predicted Label: True
Total Parameters: 30849


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


In [5]:
# Export the loaded Siamese Network model to ONNX
torch.onnx.export(loaded_model,                      # loaded Siamese Network model
                  (img1, img2),                      # model input (or a tuple for multiple inputs)
                  model_path,                        # where to save the ONNX model
                  export_params=True,                # store the trained parameter weights inside the model file
                  opset_version=14,                  # the ONNX version to export the model to
                  do_constant_folding=True,          # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})



# Move tensors from GPU to CPU
img1_cpu = img1.cpu()
img2_cpu = img2.cpu()
print(img1_cpu.size())

# Serialize data into a JSON file
input_data = [
    img1_cpu.detach().numpy().reshape([-1]).tolist(),
    img2_cpu.detach().numpy().reshape([-1]).tolist(),
]

data = dict(input_data=input_data)
json.dump(data, open(data_path, 'w'))

torch.Size([1, 1, 10, 10])


In [6]:
# *** Providing the calibrate settings function with a larger and broader range of sample inputs.


csv_path = '../training/dataset/forest_dataset.csv'
df = pd.read_csv(csv_path)

# Construct the full paths to the images
img1_files = [os.path.join('..', 'training', path) for path in df['imageT0'].iloc[:20]]
img2_files = [os.path.join('..', 'training', path) for path in df['imageT1'].iloc[:20]]

img1_list = []
img2_list = []
for img1_path, img2_path in zip(img1_files, img2_files):
    img1, img2 = siamese_loader.load_and_transform_pair(img1_path, img2_path)
    img1 = img1.unsqueeze(0).cpu().detach().numpy().reshape([-1]).tolist()
    img2 = img2.unsqueeze(0).cpu().detach().numpy().reshape([-1]).tolist()
    img1_list.extend(img1) # don't use .append bc it should look like [[xxx...],[yyy...]]
    img2_list.extend(img2)

data = dict(input_data = [img1_list, img2_list])
# Serialize data into file:
json.dump( data, open(cal_path, 'w' ))

In [7]:
#For ezkl to compute a snark, it needs some settings to determine how to create the circuit.
#This cell instantiates some parameters that determine the circuit shape, size etc
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "private"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # "fixed" for params means that the committed to params are used for all proofs
py_run_args.variables = [("batch_size", 1)]

!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
assert res == True
print("gen_settings OK")

# *** SCALES is one of main knobs you can turn to trade off accuracy for proving efficiency
# Under the hood calibration iterates over the scales array to see how precise you can go before failure. 
# For example if 7 fails it falls back to 1.
# (Right now the default scales values for the resources target is 8-10 and 10-13  for accuracy)
#res = ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources", scales = [1, 7])
res = ezkl.calibrate_settings(cal_path, model_path, settings_path, "accuracy")
assert res == True

INFO ezkl.graph.model 2024-01-08 18:58:40,694 model.rs:775 set batch_size to 1
INFO ezkl.graph.model 2024-01-08 18:58:41,053 model.rs:477 model has 1 instances
INFO ezkl.graph.model 2024-01-08 18:58:41,059 model.rs:1398 calculating num of constraints using dummy model layout...


INFO ezkl.graph.model 2024-01-08 18:58:41,780 model.rs:1473 model uses 57011 rows (coord=114023, constants=97549)
INFO ezkl.graph.model 2024-01-08 18:58:41,885 model.rs:775 set batch_size to 1


gen_settings OK


INFO ezkl.execute 2024-01-08 18:58:42,074 execute.rs:635 num of calibration batches: 20
INFO ezkl.graph.model 2024-01-08 18:58:42,086 model.rs:775 set batch_size to 1
INFO ezkl.graph.model 2024-01-08 18:58:42,380 model.rs:477 model has 1 instances
INFO ezkl.graph.model 2024-01-08 18:58:42,382 model.rs:1398 calculating num of constraints using dummy model layout...
INFO ezkl.graph.model 2024-01-08 18:58:43,084 model.rs:1473 model uses 57747 rows (coord=115495, constants=98875)
INFO ezkl.graph 2024-01-08 18:58:43,088 mod.rs:748 input scales: [10, 10]
INFO ezkl.graph.model 2024-01-08 18:58:44,029 model.rs:477 model has 1 instances
INFO ezkl.graph.model 2024-01-08 18:58:44,040 model.rs:1398 calculating num of constraints using dummy model layout...
INFO ezkl.graph.model 2024-01-08 18:58:44,570 model.rs:1473 model uses 57747 rows (coord=115495, constants=98875)
INFO ezkl.graph 2024-01-08 18:58:44,583 mod.rs:1025 setting lookup_range to: (-5428042, 6193922), setting logrows to: 24
INFO ezkl.

In [8]:
# Now compile the model into a circuit
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

INFO ezkl.graph.model 2024-01-08 19:08:16,367 model.rs:775 set batch_size to 1


In [9]:
# get public srs from kzg ceremony, saved to srs path.
res = ezkl.get_srs(settings_path)
assert res == True

INFO ezkl.execute 2024-01-08 19:08:16,799 execute.rs:469 SRS already exists at that path


In [10]:
# Setup the circuit and make sure the keys are generated afterwards.
start_time = time.time()

res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
    )

end_time = time.time()

execution_time = end_time - start_time

vk_size = os.path.getsize(vk_path)
pk_size = os.path.getsize(pk_path)

with open(test_time_and_size_path, 'a') as f:
   f.write(f"Verification key size: {vk_size} bytes\n")
   f.write(f"Proving key size: {pk_size} bytes\n")
   f.write(f"Setup time: {execution_time}\n")

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)

INFO ezkl.pfsys.srs 2024-01-08 19:08:16,908 srs.rs:23 loading srs from "/home/manu/.ezkl/srs/kzg24.srs"


INFO ezkl.execute 2024-01-08 19:08:36,907 execute.rs:1782 downsizing params to 24 logrows
INFO ezkl.graph.vars 2024-01-08 19:08:36,909 vars.rs:422 number of blinding factors: 5
INFO ezkl.graph.model 2024-01-08 19:08:36,912 model.rs:1076 configuring model
WARNING ezkl.circuit.table 2024-01-08 19:08:36,913 table.rs:167 Using 15 columns for non-linearity table.
WARNING ezkl.circuit.table 2024-01-08 19:08:36,918 table.rs:167 Using 15 columns for non-linearity table.
WARNING ezkl.circuit.table 2024-01-08 19:08:36,951 table.rs:167 Using 15 columns for non-linearity table.
INFO ezkl.graph 2024-01-08 19:08:36,957 mod.rs:1361 circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num_fixed": 61,
  "num_instances": 1,
  "num_selectors": 26
}
thread '<unnamed>' panicked at /root/.cargo/git/checkouts/halo2-049b997cf7195aea/57b9123/halo2_proofs/src/poly/domain.rs:55:9:
assertion failed: extended_k <= F::S
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: assertion failed: extended_k <= F::S

In [ ]:
# GENERATE the verifier associated with the circuit
try:
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "solc-select"])
    !solc-select install 0.8.20
    !solc-select use 0.8.20
    !solc --version
# rely on local installation if the notebook is not in colab
except:
    pass


In [ ]:
res = ezkl.create_evm_verifier(
        vk_path,
        settings_path,
        sol_code_path,
        abi_path
    )
assert res == True
assert os.path.isfile(sol_code_path)

## PROVE and VERIFY
### here we will generate and verify a proof locally. Then we will format the inputs and the proof in a way compatible for the evm verifier

In [ ]:
witness_path = os.path.join('ezkl-outputs/witness.json')
# generate the witness file
res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [ ]:
# !!!Generate the proof!!!
start_time = time.time()

proof = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        "single",
    )

end_time = time.time()

execution_time = end_time - start_time
with open(test_time_and_size_path, 'a') as f:
   f.write(f"Proving time: {execution_time}\n")

print(proof)
assert os.path.isfile(proof_path)

In [ ]:
# Sanity check off-chain verification
start_time = time.time()

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
    )

end_time = time.time()

execution_time = end_time - start_time
with open(test_time_and_size_path, 'a') as f:
   f.write(f"Verification (locally) time: {execution_time}\n")

assert res == True
print("verified locally")

In [ ]:
# Get input data for the verifier contract (evmInputs.json)
onchain_input_array = []
# avoiding printing last comma
formatted_output = "["
for i, value in enumerate(proof["instances"]):
    for j, field_element in enumerate(value):
        onchain_input_array.append(ezkl.vecu64_to_felt(field_element))
        formatted_output += str(onchain_input_array[-1])
        if j != len(value) - 1:
            formatted_output += ", "
    formatted_output += "]"
print("form output",formatted_output )
    
input_list = [int(entry, 0) for entry in formatted_output.strip('[]').split(', ')]
quoted_list = [format(entry, "#066x") for entry in input_list]
json_data = {
    "instances": quoted_list,
    "proof": "0x" + proof["proof"]
}
with open('evmInputs.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=2)
print("Saved evmInputs.json: ", json_data)
